In [1]:
lines="""System Tools:
!System_Tools.png!

Numbered Mockup:
!System_Tools_Numbered.png!

*FRS2886.1* Appearance
All Boolean buttons will be 40 x 90 pixels.
# Item 1 - a string indicator labeled "Current Bioreactor Name"
# Item 2 - a string control labeled "New Bioreactor Name"
# Item 3 - a Boolean button labeled Change
# Item 4 - a Boolean button labeled Reboot
# Item 5 - a horizontal graduated bar with no visible labels
# Item 6 - a Boolean button labeled "Sync RIO"
# Item 7 - a Boolean square LED measuring 36 x 36 pixels
# Item 8 - a Boolean button labeled Buzzer

*FRS2886.2* Functionality
# To change bioreactor name, the user first enters the "New Bioreactor Name." After pressing Item 3, the "Current Bioreactor Name" reflects the "New Bioreactor name."  
## Record a user event with the message "Bioreactor Renamed: %s" where %s is the user-entered bioreactor name.  The user event is to be written after the bioreactor name is successfully saved.  
## Users will not be able to save an empty string, or "Unnamed", as the bioreactor name.
# When user press Reboot, the graduated bar will become visible.
## It will increase based on the elapsed time/total time.
# When the user presses "Sync RIO Time," send the Atom's time in UTC to the RIO.
## Generate a user event indicating the user and time the RIO was synchronized, based on the Atom time. If the process is successful, the LED will turn bright green from dark green.
## If the LED is bright green initially, turn it dark green before performing any action.
# When the user presses Buzzer, it will stay in on mode and continue buzzing until user press it again to turn it off."""

In [2]:
import re
frs_match = re.compile(r"^(?:([\+\*]*) )?([\+\* ]*)(FRS\d+)\.?([\d\.]+)?([\+\*\:]*)?\s*(.+)?$").match
subitem_match = re.compile(r"^(?:([#]+) )").match
magic_match = re.compile(r"^@lm\:\<(.+)\>").match

class NoMatch(Exception):
    pass

class NumState():
    VALID_INDENTS = "*+"
    LINE_FRS = 0
    LINE_SUBITEM = 1
    LINE_OTHER = 2
    def __init__(self, depth=None, startval=1, indent_char="*"):
        self._startval = startval
        self.state = self._newstate()
        self.ilvl = 0
        self.nlvl = 0 
        self.clvl = 0
        self.frs = ""
        self.nums = ""
        self.fmt = ""
        self.depth = depth
        if indent_char not in self.VALID_INDENTS:
            raise ValueError("Invalid Indent Character " + indent_char)
        self.indent_char = indent_char
        self.line = ""
        
    def _newstate(self):
        return [self._startval]
        
    def _grow(self, n):
        while len(self.state) < n:
            self.state.append(self._startval)
    
    def set_state(self, lvl, val):
        self._grow(lvl)
        self.state[lvl] = val
    
    def set_state_from_nums(self, nums):
        if not nums:
            lnums = []
        else:
            lnums = nums.split(".")
        self.state = [self._startval] * len(lnums)
        for i, n in enumerate(lnums):
            self.state[i] = int(n)
        return len(lnums)
    
    def set_str(self, s):
        m = frs_match(s)
        if not m:
            raise NoMatch(s)
        return self.set_match(m)

    def set_match(self, m):
        indent, pre_fmt, frs, nums, post_fmt, bob = m.groups()
        self.ilvl = len(indent or "")
        self.fmt = pre_fmt
        self.frs = frs
        self.nlvl = self.set_state_from_nums(nums)
        self.clvl = 0
        
    def frs_string(self):
        return self.format_lvl(self.clvl)
    
    def indent(self, n=1):
        maxn = self.clvl + self.nlvl + n
        minn = self.clvl + self.nlvl
        self._grow(maxn)
        for i in range(minn, maxn):
            self.state[i] = 1
        self.clvl += n
    
    def dedent(self, n=1):
        self.clvl -= n
        self.next()
        
    def next(self):
        self.state[self.clvl+self.nlvl-1] += 1
        
    def feed(self, line):
        self.line = line
        m = frs_match(line)
        if m:
            self.set_match(m)
            return self.LINE_FRS
        m = subitem_match(line)
        if m:
            self.set_indent(len(m.group(1)))
            return self.LINE_SUBITEM
        return self.LINE_OTHER
    
    def set_indent(self, n):
        diff = n - (self.ilvl + self.clvl)
        if diff == 0:
            self.next()
        elif diff > 0:
            self.indent(diff)
        else:
            self.dedent(-diff)
            
    def curr_state(self, lvl):
        return self.state[:self.nlvl + lvl]
        
    def format_lvl(self, lvl):
        idl = lvl + self.ilvl
        if idl < 0: idl = 0
        ind = self.indent_char * idl
        if self.depth is not None and idl > self.depth:
            return ind
        space = " " if ind else ""
        frs = self.frs
        fmt = "*"
        nums = ".".join(str(n) for n in self.curr_state(lvl))
        dot = "." if nums else ""
        post = ":*"
        return "".join((ind, space, fmt, frs, dot, nums, post))
    
    def parse_text(self, text):
        lines = text.splitlines()
        for line in lines:
            typ = self.feed(line)
            if typ == self.LINE_FRS:
                v = line
            elif typ == self.LINE_SUBITEM:
                fmt = self.frs_string()
                v = " ".join((fmt, line.split(" ", 1)[1]))
            elif typ == self.LINE_OTHER:
                v = line
            else:
                raise ValueError(typ)
            yield v

In [11]:
def test_match(s):
    return frs_match(s).groups()

def assert_match(s, exp):
    m = test_match(s)
    assert m == exp, (m, exp)
    
assert_match("*FRS1234*", (None, "*", "FRS1234", None, "*", None))
assert_match("* *FRS1234*", ("*", "*", "FRS1234", None, "*", None))
assert_match("*FRS1234.5.6*", (None, "*", "FRS1234", "5.6", "*", None))
assert_match("*+FRS1234.5.6+*", (None, "*+", "FRS1234", "5.6", "+*", None))
assert_match("** *+FRS1234.5.6.7.8+*", ("**", "*+", "FRS1234", "5.6.7.8", "+*", None))
assert_match("* *FRS1234:*", ("*", "*", "FRS1234", None, ":*", None))
assert_match("* *FRS1234*:", ("*", "*", "FRS1234", None, "*:", None))
assert_match("* *FRS1234*: Bob", ("*", "*", "FRS1234", None, "*:", "Bob"))

def assert_ab(a, b):
    try:
        assert a == b, "%s != %s" % (a, b)
    except AssertionError as e:
        print(e.args)
        raise

def test_state(s):
    state = NumState()
    state.set_str(s)
    return state
    
def assert_state(s, statevals, ilvl, nlvl):
    state = test_state(s)
    assert_ab(state.ilvl, ilvl)
    assert_ab(state.nlvl, nlvl)
    assert_ab(state.state, statevals)

def test_ilvl(s, ilvl):
    state = NumState()
    state.set_str(s)
    return state
    
def assert_ilvl(s, ilvl):
    state = test_ilvl(s, ilvl)
    assert_ab(state.ilvl, ilvl)
    
def test_fmt(s):
    state = NumState()
    state.set_str(s)
    return state
    
def assert_fmt1(s, id, fmt):
    state = test_fmt(s)
    state.indent(id)
    assert_ab(state.frs_string(), fmt)
    
def assert_fmt2(s, id, fmt):
    state = test_fmt(s)
    for _ in range(id):
        state.indent(1)
    assert_ab(state.frs_string(), fmt)
    
def assert_indent(s, id, fmt):
    assert_fmt1(s, id, fmt)
    assert_fmt2(s, id, fmt)

assert_ilvl("*FRS1234*", 0)
assert_ilvl("*FRS1234.5.6*", 0)
assert_ilvl("** *FRS1234.5.6*", 2)

assert_state("*FRS1234*", [], 0, 0)
assert_state("*FRS1234.5.6*", [5, 6], 0, 2)
assert_state("** *FRS1234.5.6*", [5, 6], 2, 2)
assert_state("** *FRS1234*", [], 2, 0)

def assert_state2(s, ilvl, nlvl, clvl):
    state = test_state(s)
    st = state.state.copy()
    state.indent()
    st.append(1)
    assert_ab(state.state, st)
    assert_ab(state.ilvl, ilvl)
    assert_ab(state.nlvl, nlvl)
    assert_ab(state.clvl, clvl)

assert_state2("*FRS1234*", 0, 0, 1)
assert_state2("* *FRS1234*", 1, 0, 1)
assert_indent("*FRS1234*", 1, "* *FRS1234.1:*")
assert_indent("*FRS1234*", 1, "* *FRS1234.1:*")
assert_indent("* *FRS1234*", 1, "** *FRS1234.1:*")
assert_indent("*FRS1234.1", 1, "* *FRS1234.1.1:*")
assert_indent("*FRS1234.1", 2, "** *FRS1234.1.1.1:*")

def assert_next(s, exp):
    st = NumState()
    st.set_str(s)
    st.next()
    assert_ab(st.frs_string(), exp)

def assert_dedent(s, id, nxt, dd, fmt, typ=1):
    st = NumState()
    st.set_str(s)
    st.indent(id)
    for _ in range(nxt):
        st.next()
    if typ == 1:
        st.dedent(dd)
    else:
        for _ in range(dd):
            st.dedent(1)
    assert_ab(st.frs_string(), fmt)

assert_next("*FRS1234.1*", "*FRS1234.2:*")
assert_next("* *FRS1234.1*", "* *FRS1234.2:*")
assert_next("*FRS1234.1.2*", "*FRS1234.1.3:*")
assert_next("* *FRS1234.1.2*", "* *FRS1234.1.3:*")
assert_dedent("*FRS1234.1*", 1, 2, 1, "*FRS1234.2:*")
assert_dedent("*FRS1234.1*", 1, 1, 1, "*FRS1234.2:*")
assert_dedent("*FRS1234.1*", 2, 1, 1, "* *FRS1234.1.2:*")
assert_dedent("*FRS1234.1*", 2, 1, 1, "* *FRS1234.1.2:*")
assert_dedent("*FRS1234.1*", 2, 1, 2, "*FRS1234.2:*")
assert_dedent("*FRS1234.1*", 3, 1, 2, "* *FRS1234.1.2:*")
assert_dedent("* *FRS1234.1*", 1, 2, 1, "* *FRS1234.2:*")
assert_dedent("* *FRS1234.1*", 1, 1, 1, "* *FRS1234.2:*")
assert_dedent("* *FRS1234.1*", 2, 1, 1, "** *FRS1234.1.2:*")
assert_dedent("* *FRS1234.1*", 2, 1, 1, "** *FRS1234.1.2:*")
assert_dedent("* *FRS1234.1*", 2, 1, 2, "* *FRS1234.2:*")
assert_dedent("* *FRS1234.1*", 3, 1, 2, "** *FRS1234.1.2:*")

def assert_feed_indent(s, feed, exp):
    st = NumState()
    st.set_str(s)
    st.feed(feed)
    assert_ab(st.frs_string(), exp)
    
assert_feed_indent("*FRS1234*", "# Bob", "* *FRS1234.1:*")
assert_feed_indent("*FRS1234*", "## Bob", "** *FRS1234.1.1:*")
assert_feed_indent("*FRS1234.1*", "# Bob", "* *FRS1234.1.1:*")
assert_feed_indent("*FRS1234.1*", "## Bob", "** *FRS1234.1.1.1:*")
assert_feed_indent("* *FRS1234.1*", "# Bob", "* *FRS1234.2:*")
assert_feed_indent("* *FRS1234*", "## Bob", "** *FRS1234.1:*")
assert_feed_indent("** *FRS1234.1*", "## Bob", "** *FRS1234.2:*")
assert_feed_indent("* *FRS1234.1*", "## Bob", "** *FRS1234.1.1:*")
assert_feed_indent("** *FRS1234.1.1", "# Bob", "* *FRS1234.2:*")

def assert_double_parse(t, d=5):
    r = list(NumState(d).parse_text(t))
    r2 = list(NumState(d).parse_text("\n".join(r)))
    assert_ab(r, r2)
    
dpt = """*FRS2881.4*
# PBSUsers.conf file will keep track of the following data:
## Per User
### User Name
### Group
### Password
### Email Address
### Date Password was Saved
### Number of Failed Login Attempts
## Per Group
### Group Name
### Password Expiration Period (Days)
## 2-D Permission Table
### Each row corresponds to each group"""
assert_double_parse(dpt)

def assert_new_state(s, exp, nxt=1, ind=0):
    st = NumState()
    for l in s.splitlines():
        st.feed(l)
    for _ in range(nxt):
        st.next()
    for _ in range(ind):
        st.indent()
    assert_ab(st.frs_string(), exp)
    
assert_new_state("*FRS123.1*\n*FRS31.2*", "*FRS31.3:*")
assert_new_state("*FRS123.1*\n*FRS123.3*", "*FRS123.4:*")

In [128]:
depth=2
parsed = NumState(depth).parse_text(lines)
l = []
for l1, l2 in zip(lines.splitlines(), parsed):
    l.append(("%-25s" % l1[:20], l2[:30]))
for i in l:
    print(*i)

*+Specification+*         *+Specification+*
*User Control Panel*      *User Control Panel*
*FRS2881.0* Accessin      *FRS2881.0* Accessing
The user will click       The user will click the "Accou
# If the user has "A      * *FRS2881.0.1:* If the user h
# If the user does n      * *FRS2881.0.2:* If the user d
                          
*FRS2881.1* Configur      *FRS2881.1* Configure Users ta
# Mockup                  * *FRS2881.1.1:* Mockup
!Configure_Users.png      !Configure_Users.png!
# Numbered mockup         * *FRS2881.1.2:* Numbered mock
!Configure_Users_Num      !Configure_Users_Numbered.png!
                          
*FRS2881.11* Configu      *FRS2881.11* Configure Users t
# Item 1 - Configure      * *FRS2881.11.1:* Item 1 - Con
## This will be a ta      ** *FRS2881.11.1.1:* This will
## It will be labele      ** *FRS2881.11.1.2:* It will b
## It will be the le      ** *FRS2881.11.1.3:* It will b
# Item 2 - Configure      * *FRS2881.11.2:* Item 2 - Con
# Item 3 - User List     

In [14]:
lines="""
About Panel:
!About.png!
Numbered Mockup:
!About_Numbered.png!

*FRS2891.1* Appearance
* Item 1 is a string indicator labeled Bioreactor Model
* Item 2 is a string indicator labeled RIO Version
* Item 3 is a string indicator labeled Shell Version
* Item 4 is a string indicator labeled Serial Number
* Item 5 is a string indicator labeled Startup Status
* Item 6 is a string indicator labeled DB Logger Version
* Item 7 is a string indicator labeled Network Version
* Item 8 is a string indicator labeled Database Version
* Item 9 is a string indicator labeled Webservice Version
* Item 10 is a string indicator labeled Config Status
* Item 11 is a string indicator labeled Computer IP Address
* Item 12 is a string indicator labeled Serial Number
* Item 13 is a string indicator labeled Product Number
* Item 14 is a string indicator labeled Expiration Date
* Item 15 is a table indicator. Rows are as follows:
** Name
** Start Time
** End Time
** Time Deleted
** User
** Checksum

*FRS2891.2* Functionality
# Item 1 will show the bioreactor model from the bioreactor models.cfg file on the RIO.
# Items 2, 3, 6, 7, 8, and 9 will show version numbers:
** All version numbers will start with "V"
** The number will be in dotted number format (XX.XX.XX), with 2-4 groups of numbers.
# Item 4 will show the RIO serial number. 
# Item 5 will show the current startup status.
# Item 10 will show the configuration status.
# Item 11 will show the RIO's IP Address. 
# Items 12-14 will show their respective values from the bag info file. 
# Item 15 will show the information stored in the Database History File.

"""
res = "\n".join(NumState(3).parse_text(lines))
print(res)
import clipboard
clipboard.copy(res)


About Panel:
!About.png!
Numbered Mockup:
!About_Numbered.png!

*FRS2891.1* Appearance
* Item 1 is a string indicator labeled Bioreactor Model
* Item 2 is a string indicator labeled RIO Version
* Item 3 is a string indicator labeled Shell Version
* Item 4 is a string indicator labeled Serial Number
* Item 5 is a string indicator labeled Startup Status
* Item 6 is a string indicator labeled DB Logger Version
* Item 7 is a string indicator labeled Network Version
* Item 8 is a string indicator labeled Database Version
* Item 9 is a string indicator labeled Webservice Version
* Item 10 is a string indicator labeled Config Status
* Item 11 is a string indicator labeled Computer IP Address
* Item 12 is a string indicator labeled Serial Number
* Item 13 is a string indicator labeled Product Number
* Item 14 is a string indicator labeled Expiration Date
* Item 15 is a table indicator. Rows are as follows:
** Name
** Start Time
** End Time
** Time Deleted
** User
** Checksum

*FRS2891.2* Func